# Cat vs Dog Image Classifier

This notebook implements a CNN to classify images as either cats or dogs using the Microsoft Cats vs Dogs dataset.


In [ ]:
!pip install kagglehub tensorflow

In [ ]:
import tensorflow as tf
import os
import pathlib
import kagglehub

/Users/dimitridumont/.pyenv/versions/3.11.10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Download Dataset
Now we'll download the Cats vs Dogs dataset using kagglehub:

In [ ]:
# Download latest version of the dataset
path = kagglehub.dataset_download(".")
print("Path to dataset files:", path)

## Model Implementation
Let's create our PetClassifier class that will handle the data pipeline and model training:

In [ ]:
class PetClassifier:
    def __init__(self, data_dir='PetImages', img_size=224, batch_size=32):
        self.data_dir = pathlib.Path(data_dir)
        self.img_size = img_size
        self.batch_size = batch_size
        self.setup_hardware()

    def setup_hardware(self):
        """Configure TensorFlow for available hardware."""
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                tf.keras.mixed_precision.set_global_policy('mixed_float16')
                print(f"Training on {len(gpus)} GPU(s) with mixed precision")
            except RuntimeError as e:
                print(f"GPU setup error: {e}")
                print("Training on CPU")
        else:
            print("No GPU found. Training on CPU")

    def prepare_dataset(self):
        """Create TensorFlow dataset from directory structure."""
        # Create dataset
        data = tf.keras.utils.image_dataset_from_directory(
            self.data_dir,
            validation_split=0.2,
            subset="training",
            seed=123,
            image_size=(self.img_size, self.img_size),
            batch_size=self.batch_size
        )
        
        val_data = tf.keras.utils.image_dataset_from_directory(
            self.data_dir,
            validation_split=0.2,
            subset="validation",
            seed=123,
            image_size=(self.img_size, self.img_size),
            batch_size=self.batch_size
        )

        # Configure dataset for performance
        AUTOTUNE = tf.data.AUTOTUNE
        data = data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
        val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)
        
        return data, val_data

    def build_model(self):
        """Create the CNN model."""
        model = tf.keras.Sequential([
            # Input layer
            tf.keras.layers.Input(shape=(self.img_size, self.img_size, 3)),
            
            # First conv block
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            # Second conv block
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            # Third conv block
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            # Dense layers
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
        ])
        
        return model

    def train(self, epochs=10):
        """Train the model."""
        # Prepare data
        train_ds, val_ds = self.prepare_dataset()
        
        # Build and compile model
        model = self.build_model()
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )
        
        # Callbacks
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                'best_model.h5',
                save_best_only=True,
                monitor='val_accuracy'
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor='val_accuracy',
                patience=5,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=3
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir='./logs',
                histogram_freq=1
            )
        ]
        
        # Train
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=callbacks
        )
        
        return model, history

## Training the Model
Now let's initialize our classifier and train it:

In [ ]:
# Initialize classifier with downloaded dataset path
classifier = PetClassifier(data_dir=path)

# Train the model
model, history = classifier.train(epochs=10)

## Visualize Training Results
Let's plot the training history:

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

## Save the Model
Save the trained model for later use:

In [ ]:
model.save('pet_classifier_model.h5')
print("Model saved successfully!")